# Suggestion Writer in Python

This code is main edited by Enze Wang

Fixed by Yicen Liu and Hua Tong

# Attention

Please put four .csv file in data folder and put this code in main folder instead of code folder.

This code want to write suggesion for business from the analysis above.  



In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import itertools
from collections import Counter
from nltk import FreqDist
from collections import Counter
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')
from wordcloud import WordCloud
from nltk import sent_tokenize, word_tokenize, PorterStemmer, pos_tag 
from nltk.corpus import stopwords
from nltk.text import TextCollection
from nltk.corpus import wordnet
import nltk.tokenize as nt
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

[nltk_data] Downloading package stopwords to /Users/enze/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/enze/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/enze/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/enze/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
Noun_results=pd.read_csv("data/Results_Analysis_Noun_Class.csv")
Review_results=pd.read_csv("data/Results_Analysis_Noun_Attitude.csv")
pizza_review=pd.read_csv("data/pizza_review.csv")
pizza_business=pd.read_csv("data/pizza_business.csv")
pizza_business_clean_review=pd.read_csv("data/pizza_business_clean_review.csv")
Word_Review_Matrix=pd.read_csv("data/Word_Review_Matrix.csv")

# Business Suggestion

Suggestion for business is simple. For example, if one business offer Alcohol, we can suggest them to keep. If not, suggest them to offer alcohol as possible.  

Therefore in our suggestion here, we use 1 as the business rating is high option and 0 as the low option. The detailed suggestion will raise by Shiny app code. 

For np.nan in the value, we default give low option rating suggestion. 

Some special category, such as Delivery, we will mention and give warning to those given delivery to raise their delivery quality and service instead of cancel delivery.

In [3]:
pizza_business["Alcohol"]

0                None
1                 NaN
2                None
3       beer_and_wine
4       beer_and_wine
            ...      
1555             None
1556             None
1557             None
1558             None
1559         full_bar
Name: Alcohol, Length: 1560, dtype: object

In [4]:
set(pizza_business["Alcohol"])

{'None', 'beer_and_wine', 'full_bar', nan}

In [5]:
Alcohol_Suggestion=pizza_business["Alcohol"]
Alcohol_Suggestion=Alcohol_Suggestion.replace(['beer_and_wine','full_bar'],"1")
Alcohol_Suggestion=Alcohol_Suggestion.replace(['None',np.nan],"0")

In [6]:
Alcohol_Suggestion

0       0
1       0
2       0
3       1
4       1
       ..
1555    0
1556    0
1557    0
1558    0
1559    1
Name: Alcohol, Length: 1560, dtype: object

In [7]:
pizza_business["BikeParking"]

0        True
1         NaN
2        True
3         NaN
4        True
        ...  
1555      NaN
1556    False
1557    False
1558     True
1559     True
Name: BikeParking, Length: 1560, dtype: object

In [8]:
set(pizza_business["BikeParking"])

{nan, False, True}

In [9]:
BikeParking_Suggestion=pizza_business["BikeParking"]
BikeParking_Suggestion=BikeParking_Suggestion.replace([True],"1")
BikeParking_Suggestion=BikeParking_Suggestion.replace([False,np.nan],"0")
BikeParking_Suggestion

0       1
1       0
2       1
3       0
4       1
       ..
1555    0
1556    0
1557    0
1558    1
1559    1
Name: BikeParking, Length: 1560, dtype: object

In [10]:
pizza_business["GoodForKids"]

0       True
1       True
2       True
3       True
4       True
        ... 
1555    True
1556    True
1557    True
1558    True
1559    True
Name: GoodForKids, Length: 1560, dtype: object

In [11]:
set(pizza_business["GoodForKids"])

{nan, False, True}

In [12]:
GoodForKids_Suggestion=pizza_business["GoodForKids"]
GoodForKids_Suggestion=GoodForKids_Suggestion.replace([True],"1")
GoodForKids_Suggestion=GoodForKids_Suggestion.replace([False,np.nan],"0")
GoodForKids_Suggestion

0       1
1       1
2       1
3       1
4       1
       ..
1555    1
1556    1
1557    1
1558    1
1559    1
Name: GoodForKids, Length: 1560, dtype: object

In [13]:
set(GoodForKids_Suggestion)

{'0', '1'}

In [14]:
pizza_business["NoiseLevel"]

0           quiet
1           quiet
2           quiet
3           quiet
4         average
          ...    
1555          NaN
1556      average
1557          NaN
1558      average
1559    very_loud
Name: NoiseLevel, Length: 1560, dtype: object

In [15]:
set(pizza_business["NoiseLevel"])

{'average', 'loud', nan, 'quiet', 'very_loud'}

In [16]:
NoiseLevel_Suggestion=pizza_business["NoiseLevel"]
NoiseLevel_Suggestion=NoiseLevel_Suggestion.replace(['average','quiet'],"1")
NoiseLevel_Suggestion=NoiseLevel_Suggestion.replace(['very_loud',np.nan,'loud'],"0")
NoiseLevel_Suggestion

0       1
1       1
2       1
3       1
4       1
       ..
1555    0
1556    1
1557    0
1558    1
1559    0
Name: NoiseLevel, Length: 1560, dtype: object

In [17]:
set(NoiseLevel_Suggestion)

{'0', '1'}

In [18]:
pizza_business["RestaurantsGoodForGroups"]

0        True
1       False
2        True
3        True
4        True
        ...  
1555     True
1556     True
1557     True
1558     True
1559     True
Name: RestaurantsGoodForGroups, Length: 1560, dtype: object

In [19]:
set(pizza_business["RestaurantsGoodForGroups"])

{False, True, nan}

In [20]:
RestaurantsGoodForGroups_Suggestion=pizza_business["RestaurantsGoodForGroups"]
RestaurantsGoodForGroups_Suggestion=RestaurantsGoodForGroups_Suggestion.replace([True],"1")
RestaurantsGoodForGroups_Suggestion=RestaurantsGoodForGroups_Suggestion.replace([False,np.nan],"0")
RestaurantsGoodForGroups_Suggestion

0       1
1       0
2       1
3       1
4       1
       ..
1555    1
1556    1
1557    1
1558    1
1559    1
Name: RestaurantsGoodForGroups, Length: 1560, dtype: object

In [21]:
set(RestaurantsGoodForGroups_Suggestion)

{'0', '1'}

In [22]:
pizza_business["RestaurantsReservations"]

0        True
1       False
2       False
3       False
4       False
        ...  
1555    False
1556    False
1557    False
1558    False
1559    False
Name: RestaurantsReservations, Length: 1560, dtype: object

In [23]:
set(pizza_business["RestaurantsReservations"])

{False, True, nan}

In [24]:
RestaurantsReservations_Suggestion=pizza_business["RestaurantsReservations"]
RestaurantsReservations_Suggestion=RestaurantsReservations_Suggestion.replace([True],"1")
RestaurantsReservations_Suggestion=RestaurantsReservations_Suggestion.replace([False,np.nan],"0")
RestaurantsReservations_Suggestion

0       1
1       0
2       0
3       0
4       0
       ..
1555    0
1556    0
1557    0
1558    0
1559    0
Name: RestaurantsReservations, Length: 1560, dtype: object

In [25]:
set(RestaurantsReservations_Suggestion)

{'0', '1'}

In [26]:
pizza_business["WiFi"]

0         no
1         no
2         no
3         no
4       free
        ... 
1555      no
1556      no
1557    free
1558      no
1559      no
Name: WiFi, Length: 1560, dtype: object

In [27]:
set(pizza_business["WiFi"])

{'free', nan, 'no', 'paid'}

In [28]:
WiFi_Suggestion=pizza_business["WiFi"]
WiFi_Suggestion=WiFi_Suggestion.replace(['free'],"1")
WiFi_Suggestion=WiFi_Suggestion.replace(['no','paid',np.nan],"0")
WiFi_Suggestion

0       0
1       0
2       0
3       0
4       1
       ..
1555    0
1556    0
1557    1
1558    0
1559    0
Name: WiFi, Length: 1560, dtype: object

In [29]:
set(WiFi_Suggestion)

{'0', '1'}

In [30]:
pizza_business["RestaurantsDelivery"]

0        True
1        True
2        True
3        True
4        True
        ...  
1555      NaN
1556     True
1557    False
1558     True
1559     True
Name: RestaurantsDelivery, Length: 1560, dtype: object

In [31]:
set(pizza_business["RestaurantsDelivery"])

{False, True, nan}

In [32]:
RestaurantsDelivery_Suggestion=pizza_business["RestaurantsDelivery"]
RestaurantsDelivery_Suggestion=RestaurantsDelivery_Suggestion.replace([False],"1")
RestaurantsDelivery_Suggestion=RestaurantsDelivery_Suggestion.replace([True,np.nan],"0")
RestaurantsDelivery_Suggestion

0       0
1       0
2       0
3       0
4       0
       ..
1555    0
1556    0
1557    1
1558    0
1559    0
Name: RestaurantsDelivery, Length: 1560, dtype: object

In [33]:
set(RestaurantsDelivery_Suggestion)

{'0', '1'}

In [34]:
pizza_business["BusinessParking"]

0          NaN
1          NaN
2          NaN
3          lot
4          lot
         ...  
1555       NaN
1556       NaN
1557       NaN
1558    street
1559       NaN
Name: BusinessParking, Length: 1560, dtype: object

In [35]:
Park=list(set(pizza_business["BusinessParking"]))
Park=[x for x in Park if str(x) != 'nan']
Park

['street lot',
 'street',
 'garage street',
 'garage street validated',
 'garage valet street',
 'lot',
 'valet street',
 'valet',
 'garage valet street validated',
 'valet validated',
 'garage',
 'validated',
 'valet lot',
 'lot validated',
 'garage street lot']

In [36]:
BusinessParking_Suggestion=pizza_business["BusinessParking"]
BusinessParking_Suggestion=BusinessParking_Suggestion.replace(Park,"1")
BusinessParking_Suggestion=BusinessParking_Suggestion.replace([np.nan],"0")
BusinessParking_Suggestion

0       0
1       0
2       0
3       1
4       1
       ..
1555    0
1556    0
1557    0
1558    1
1559    0
Name: BusinessParking, Length: 1560, dtype: object

In [37]:
set(BusinessParking_Suggestion)

{'0', '1'}

In [38]:
Business_Suggestion=pd.DataFrame([pizza_business["business_id"],
                                  Alcohol_Suggestion,
BikeParking_Suggestion,
GoodForKids_Suggestion,
NoiseLevel_Suggestion,
RestaurantsGoodForGroups_Suggestion,
RestaurantsReservations_Suggestion,
WiFi_Suggestion,
RestaurantsDelivery_Suggestion,
BusinessParking_Suggestion])
Business_Suggestion=Business_Suggestion.transpose()
Business_Suggestion

,business_id,Alcohol,BikeParking,GoodForKids,NoiseLevel,RestaurantsGoodForGroups,RestaurantsReservations,WiFi,RestaurantsDelivery,BusinessParking
0,ZkzutF0P_u0C0yTulwaHkA,0,1,1,1,1,1,0,0,0
1,39lLJK_rrYY2NYomSsQdUA,0,0,1,1,0,0,0,0,0
2,0y6alZmSLnPzmG5_kP5Quw,0,1,1,1,1,0,0,0,0
3,_Kp1IPTi17wBywYOd30raA,1,0,1,1,1,0,0,0,1
4,crBw_Hq3gCkppirgjnIl4A,1,1,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,0,0,1,0,1,0,0,0,0
1556,ZZGs9idRiKRHue2RyXw9uA,0,0,1,1,1,0,0,0,0
1557,v8GgrqLTs_NZZl2tkyvOgA,0,0,1,0,1,0,1,1,0
1558,qWcaGio9JsqrvgmQiIRtIQ,0,1,1,1,1,0,0,0,1


Check the number of 1 according to with 8_Analysis_Business

In [39]:
Counter(Business_Suggestion["Alcohol"])

Counter({'0': 1085, '1': 475})

In [40]:
Counter(Business_Suggestion["BikeParking"])

Counter({'1': 688, '0': 872})

In [41]:
Counter(Business_Suggestion["GoodForKids"])

Counter({'1': 1094, '0': 466})

In [42]:
Counter(Business_Suggestion["NoiseLevel"])

Counter({'1': 861, '0': 699})

In [43]:
Counter(Business_Suggestion["RestaurantsDelivery"])

Counter({'0': 1209, '1': 351})

In [44]:
Counter(Business_Suggestion["RestaurantsGoodForGroups"])

Counter({'1': 964, '0': 596})

In [45]:
Counter(Business_Suggestion["RestaurantsReservations"])

Counter({'1': 289, '0': 1271})

In [46]:
Counter(Business_Suggestion["WiFi"])

Counter({'0': 1131, '1': 429})

In [47]:
Counter(Business_Suggestion["BusinessParking"])

Counter({'0': 823, '1': 737})

Check, the business_id 0y6alZmSLnPzmG5_kP5Quw

In [48]:
Business_Suggestion.iloc[2]

business_id                 0y6alZmSLnPzmG5_kP5Quw
Alcohol                                          0
BikeParking                                      1
GoodForKids                                      1
NoiseLevel                                       1
RestaurantsGoodForGroups                         1
RestaurantsReservations                          0
WiFi                                             0
RestaurantsDelivery                              0
BusinessParking                                  0
Name: 2, dtype: object

In pizza_business:

In [49]:
pizza_business.iloc[2]

Unnamed: 0                                              2
business_id                        0y6alZmSLnPzmG5_kP5Quw
name                                          J J's Pizza
address                                   20542 Lorain Rd
city                                            Cleveland
latitude                                          41.4483
longitude                                        -81.8476
stars                                                 4.5
review_count                                           21
categories                    Pizza, Italian, Restaurants
Alcohol                                              None
Ambience                                              NaN
BikeParking                                          True
BusinessAcceptsCreditCards                           True
Caters                                               True
GoodForKids                                          True
HasTV                                                True
NoiseLevel    

No problem

# Noun suggestion

The noun suggestion suggest business to offer or add XXX service to their business, something like food or service. We will raise suggestion like:

Review contains XXX, XXX and XXX individually always have high review star rating compared with that without. Think about add word related service or food to increase your star rating. 

or

Review contains XXX, XXX and XXX individually always have low review star rating compared with that without. Think about remove or fix word related service or food to increase your star rating. 

The Noun suggestion is the most difficult part in our suggestion, because:

1. Some review contains "cheese" for example may something like "I want you offer cheese pizza". Therefore it is strange to provide suggestion like "Keep cheese in your food list" if this business review does not contain "cheese" but it does offer cheese.

Our suggestion will be divided into four part. Add, Keep, Fix, Avoid. 

Add means that you do not have positive noun and we suggest you add. 

Keep means that you do have positive noun and we suggest you keep. 

Fix means that you do have negative noun and we suggest you fix. 

Avoid means that you do not have negative noun and we suggest you avoid. 

In [50]:
Noun_results=Noun_results.rename(columns={"Unnamed: 0": "Index"})
Noun_results

,Index,Food,Lasso_Coefficients
0,1,(Intercept),3.642675
1,4,beer,0.042015
2,7,wine,0.073241
3,8,sausage,0.039468
4,14,bacon,0.073563
5,15,mozzarella,0.109189
6,17,shrimp,-0.056449
7,18,fish,-0.047833
8,19,marinara,-0.006065
9,20,spaghetti,-0.038752


In [51]:
New_Noun_results=Noun_results.drop([0])
New_Noun_results

,Index,Food,Lasso_Coefficients
1,4,beer,0.042015
2,7,wine,0.073241
3,8,sausage,0.039468
4,14,bacon,0.073563
5,15,mozzarella,0.109189
6,17,shrimp,-0.056449
7,18,fish,-0.047833
8,19,marinara,-0.006065
9,20,spaghetti,-0.038752
10,21,lettuce,-0.550178


In [52]:
Positive_Noun=New_Noun_results[New_Noun_results["Lasso_Coefficients"]>0]["Food"].tolist()
Negative_Noun=New_Noun_results[New_Noun_results["Lasso_Coefficients"]<0]["Food"].tolist()
Positive_Noun
Negative_Noun

['shrimp',
 'fish',
 'marinara',
 'spaghetti',
 'lettuce',
 'caesar',
 'noodle',
 'tea',
 'broccoli',
 'wing',
 'coke',
 'patty',
 'cracker']

In [53]:
pizza_business_clean_review

,Unnamed: 0,review_id,clean_review
0,0,hfXVxrlD6fr1kXfz6sR13Q,parking tough get dont get go much also get li...
1,1,EDQpLLFiAxeSxWI5ipwzBQ,server extensive knowledge wine various menu s...
2,2,RWmT90rP7wYo57dG1cyvww,fast forward every trip nyc pitt includes visi...
3,3,0jduztItUhOh3FzKGjMxZQ,far favorite clevelandakron restaurant great f...
4,4,sxfmzhMvZ8LlI1x4fhNjKg,food good usually get italian sub pizza would ...
...,...,...,...
67151,67151,gcNRaJMDlSVs8YlqwpsRuw,im originally side ohio ive always really enjo...
67152,67152,yFxDOdiDb8HPaK1io81kiw,yes yes yes loved going place shadyside ecstat...
67153,67153,Z_xmoLgIMB4As90t0a5eBQ,went lunch meeting amidst typical suburban str...
67154,67154,CzYRFCnkbqOVb3WIMMrbfw,wife eaten la brioche least dozen time food ne...


In [54]:
pizza_business_review=pd.merge(pizza_review, pizza_business_clean_review, how='left', on='review_id')

In [55]:
pizza_business_review

,Unnamed: 0_x,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Unnamed: 0_y,clean_review
0,0,hfXVxrlD6fr1kXfz6sR13Q,7nIzgNF7YzI-UAyYr3y9hw,J9f-9Prw2YVM-fiZqv2fmQ,2,0,1,1,Parking is tough to get here so I don't get to...,2013-09-04 04:35:27,0,parking tough get dont get go much also get li...
1,1,EDQpLLFiAxeSxWI5ipwzBQ,SkueY7hVsyXmYItELoodGw,lpel3pRFAzlow7u2TJMz4Q,4,3,0,0,The server had an extensive knowledge of wines...,2015-04-28 19:28:12,1,server extensive knowledge wine various menu s...
2,2,RWmT90rP7wYo57dG1cyvww,n3ahpmnEXsZYj3nbVMoUmg,Zja7ykq3icDzkN20XX6j1w,5,1,0,1,Fast Forward\n\nEvery trip from NYC to Pitt in...,2013-07-12 23:08:35,2,fast forward every trip nyc pitt includes visi...
3,3,0jduztItUhOh3FzKGjMxZQ,lVbPEaNUxFjwUydjentrYw,hr0vxeTzlHx6fTpSSmu2AA,5,1,0,0,By far my favorite Cleveland/Akron restaurant!...,2013-12-07 01:09:23,3,far favorite clevelandakron restaurant great f...
4,4,sxfmzhMvZ8LlI1x4fhNjKg,SQKRGzDa7qd8qHb13Z2-Hg,F4kjKxXXyn3e-BYVb1mWcg,4,0,0,0,All food is good here I usually get the Italia...,2014-10-27 01:42:40,4,food good usually get italian sub pizza would ...
...,...,...,...,...,...,...,...,...,...,...,...,...
67151,67151,gcNRaJMDlSVs8YlqwpsRuw,_Rgn4SZT2wXyQnW9AD9Tog,nR3u-wQ99bop6IdGoU38KQ,1,1,0,0,I'm originally from the other side of Ohio and...,2019-09-21 00:24:03,67151,im originally side ohio ive always really enjo...
67152,67152,yFxDOdiDb8HPaK1io81kiw,4r33dXcE1oYZxjONrhxTiA,yT1Ad3auuw1jAJEZHr7Yqw,5,1,1,1,"Yes, yes, YES!! We loved going to the place in...",2019-09-03 04:32:16,67152,yes yes yes loved going place shadyside ecstat...
67153,67153,Z_xmoLgIMB4As90t0a5eBQ,Iy6HkAzFcpeU7GF0e1FRSA,R13VkcyUIU3xG2fRFbBvBA,4,3,3,3,Went here for a lunch meeting amidst s typical...,2019-12-11 16:18:26,67153,went lunch meeting amidst typical suburban str...
67154,67154,CzYRFCnkbqOVb3WIMMrbfw,TP9Tz7gcAfK-BeT16qpZ2w,M-caugKzJ9qLOmCBfD9Nyg,1,1,0,0,My wife and I have eaten at La Brioche at leas...,2017-06-13 02:14:53,67154,wife eaten la brioche least dozen time food ne...


In [56]:
business_review=pizza_business["business_id"]

In [57]:
ID=[]
Add=[]
Keep=[]
Fix=[]
Avoid=[]
for i in business_review.tolist():
    ID.append(i)
    business_add=[]
    business_keep=[]
    business_fix=[]
    business_avoid=[]
    target_review=" ".join(pizza_business_review[pizza_business_review["business_id"]==i]["clean_review"].tolist())
    for j in Positive_Noun:
        if j in target_review.split(" "):
            business_keep.append(j)
        else:
            business_add.append(j)
    for j in Negative_Noun:
        if j in target_review.split(" "):
            business_fix.append(j)
        else:
            business_avoid.append(j)   
            
    
    Add.append(" ".join(business_add))
    Keep.append(" ".join(business_keep))
    Fix.append(" ".join(business_fix))
    Avoid.append(" ".join(business_avoid))
    

In [58]:
Noun_Suggestion=pd.DataFrame([ID,Add,Keep,Fix,Avoid]).transpose()

In [59]:
Noun_Suggestion=Noun_Suggestion.rename(columns={0: "business_id", 1: "Add", 2: "Keep", 3: "Fix", 4: "Avoid"})

In [60]:
Noun_Suggestion

,business_id,Add,Keep,Fix,Avoid
0,ZkzutF0P_u0C0yTulwaHkA,beer wine mozzarella bbq gelato cocktail pesto...,sausage bacon,lettuce wing,shrimp fish marinara spaghetti caesar noodle t...
1,39lLJK_rrYY2NYomSsQdUA,wine sausage bacon mozzarella bbq gelato cockt...,beer,tea wing cracker,shrimp fish marinara spaghetti lettuce caesar ...
2,0y6alZmSLnPzmG5_kP5Quw,wine bacon mozzarella bbq gelato cocktail pest...,beer sausage,shrimp fish marinara wing,spaghetti lettuce caesar noodle tea broccoli c...
3,_Kp1IPTi17wBywYOd30raA,beer wine sausage bacon mozzarella gelato cock...,bbq salami,wing,shrimp fish marinara spaghetti lettuce caesar ...
4,crBw_Hq3gCkppirgjnIl4A,wine bacon bbq gelato cocktail pesto margherit...,beer sausage mozzarella artichoke margarita sa...,marinara spaghetti lettuce caesar,shrimp fish noodle tea broccoli wing coke patt...
...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,beer wine sausage gelato cocktail margherita c...,bacon mozzarella bbq pesto crunchy arugula art...,marinara cracker,shrimp fish spaghetti lettuce caesar noodle te...
1556,ZZGs9idRiKRHue2RyXw9uA,beer wine sausage mozzarella bbq gelato cockta...,bacon,,shrimp fish marinara spaghetti lettuce caesar ...
1557,v8GgrqLTs_NZZl2tkyvOgA,beer sausage bacon mozzarella bbq gelato cockt...,wine,lettuce,shrimp fish marinara spaghetti caesar noodle t...
1558,qWcaGio9JsqrvgmQiIRtIQ,beer wine bacon mozzarella gelato cocktail pes...,sausage bbq crunchy artichoke cauliflower,caesar broccoli wing,shrimp fish marinara spaghetti lettuce noodle ...


Check. For example, the business_id ZkzutF0P_u0C0yTulwaHkA, its review is:

In [61]:
target_review=" ".join(pizza_business_review[pizza_business_review["business_id"]=="ZkzutF0P_u0C0yTulwaHkA"]["clean_review"].tolist())

In [62]:
target_review

'stopped wife due pizza sign front didnt disappoint staff amazingly friendly upon placing order asked type sauce wanted said oh havent said hold went back brought cup sauce roll cut try sauce customer service alone place remarkable pizza came wow huge crispy wonderful also got hoagie bread amazing cooked perfectly new go pizza spot plum area favor check also try sweet sauce amazing sweet sauce awesome friendly service honestly favorite pizza joint ever keep good work place look little dirty pizza crust soggy sucked italian hoagie ok nothing speciali wouldnt go back one guy answering phone ringing order cooking without washing hand time good pizza super excited find great pizza close home delivery door lazy deliver appreciate new pizza place star rating yum seven year hearing great place wasis family decided give try boy disappointed ordered large one topping pizza sweet sauce disappoint extremely flavorful tasty crust seasoned hint garlic mix best world eating pizza yummy breadstick fi

In [63]:
"beer" in target_review.split(" ")

False

In [64]:
"sausage" in target_review.split(" ")

True

In [65]:
"lettuce" in target_review.split(" ")

True

In [66]:
"shrimp" in target_review.split(" ")

False

No problem. 

# Review suggestion

In review suggestion, we will calcualte the positive review percentage of a word by business

In [67]:
Review_results

,Unnamed: 0,Word,T_Test_P_Value,T_Sig,Mean_Difference
0,1,pizza,0.000000e+00,True,1.672608
1,2,food,0.000000e+00,True,1.942552
2,3,cheese,1.727939e-75,True,1.192100
3,4,sauce,5.410469e-88,True,1.213220
4,5,service,0.000000e+00,True,2.341849
...,...,...,...,...,...
105,106,topping,1.363301e-43,True,1.334586
106,107,dressing,1.380260e-22,True,1.448045
107,108,olive,3.728876e-05,True,0.768617
108,109,serving,3.463864e-16,True,1.521354


In [68]:
Review_Word=Review_results["Word"].tolist()
Review_Word

['pizza',
 'food',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'menu',
 'delivery',
 'staff',
 'price',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'server',
 'flavor',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'location',
 'fry',
 'meatball',
 'waitress',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'manager',
 'style',
 'mushroom',
 'pepper',
 'dough',
 'dessert',
 'soup',
 'steak',
 'onion',
 'bartender',
 'bacon',
 'kitchen',
 'hoagie',
 'fan',
 'parking',
 'mozzarella',
 'employee',
 'water',
 'potato',
 'waiter',
 'entree',
 'oil',
 'cream',
 'serve',
 'shrimp',
 'fish',
 'parmesan',
 'spinach',
 'beef',
 'egg',
 'brunch',
 'ice',
 'chef',
 'breadstick',
 'breakfast',
 'cake',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'bbq',
 'music',
 'lasagna',
 'cocktail',
 'eggplant',
 'gnocchi',
 'butter',
 'ravioli',
 'taco',
 'margherita',
 'chocolate',
 'coffee',
 'takeout',
 'management',
 'building',


In [69]:
A=Review_Word
B=["review_id","business_id","stars"]
B.extend(A)
B

['review_id',
 'business_id',
 'stars',
 'pizza',
 'food',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'menu',
 'delivery',
 'staff',
 'price',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'server',
 'flavor',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'location',
 'fry',
 'meatball',
 'waitress',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'manager',
 'style',
 'mushroom',
 'pepper',
 'dough',
 'dessert',
 'soup',
 'steak',
 'onion',
 'bartender',
 'bacon',
 'kitchen',
 'hoagie',
 'fan',
 'parking',
 'mozzarella',
 'employee',
 'water',
 'potato',
 'waiter',
 'entree',
 'oil',
 'cream',
 'serve',
 'shrimp',
 'fish',
 'parmesan',
 'spinach',
 'beef',
 'egg',
 'brunch',
 'ice',
 'chef',
 'breadstick',
 'breakfast',
 'cake',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'bbq',
 'music',
 'lasagna',
 'cocktail',
 'eggplant',
 'gnocchi',
 'butter',
 'ravioli',
 'taco',
 'margherita',
 'chocolate',
 'coffee',


In [70]:
Col=B

In [71]:
Word_Review_Matrix_New=Word_Review_Matrix[Col]
Word_Review_Matrix_New

,review_id,business_id,stars,pizza,food,cheese,sauce,service,salad,restaurant,...,doughy,cooking,diner,fee,wing,topping,dressing,olive,serving,seasoning
0,hfXVxrlD6fr1kXfz6sR13Q,J9f-9Prw2YVM-fiZqv2fmQ,2,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EDQpLLFiAxeSxWI5ipwzBQ,lpel3pRFAzlow7u2TJMz4Q,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RWmT90rP7wYo57dG1cyvww,Zja7ykq3icDzkN20XX6j1w,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0jduztItUhOh3FzKGjMxZQ,hr0vxeTzlHx6fTpSSmu2AA,5,NaN,1.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sxfmzhMvZ8LlI1x4fhNjKg,F4kjKxXXyn3e-BYVb1mWcg,4,1.0,1.0,NaN,-1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67151,gcNRaJMDlSVs8YlqwpsRuw,nR3u-wQ99bop6IdGoU38KQ,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67152,yFxDOdiDb8HPaK1io81kiw,yT1Ad3auuw1jAJEZHr7Yqw,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67153,Z_xmoLgIMB4As90t0a5eBQ,R13VkcyUIU3xG2fRFbBvBA,4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67154,CzYRFCnkbqOVb3WIMMrbfw,M-caugKzJ9qLOmCBfD9Nyg,1,NaN,1.0,NaN,NaN,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
business_review=pizza_business["business_id"]

In [73]:
ID=[]
Word=[]
Business_Review_Word=[]
for i in business_review.tolist():
    ID.append(i)
    Word_Results=[]
    WordList=[]
    Target_Review=Word_Review_Matrix_New[Word_Review_Matrix_New["business_id"]==i]
    for j in Review_Word:
        results=" "
        Positive_Review_Num=Target_Review[j].tolist().count(1)
        Negative_Review_Num=Target_Review[j].tolist().count(-1)
        Neutral_Review_Num=Target_Review[j].tolist().count(0)
        Sum=Positive_Review_Num+Negative_Review_Num+Neutral_Review_Num
        # Only consider the sum of review larger than 5. 
        if Sum>5:
            results=j+" "+str(Positive_Review_Num)+" "+str(Neutral_Review_Num)+" "+str(Negative_Review_Num)
            Word_Results.append(results)
            WordList.append(j)    
    Word.append(";".join(Word_Results))
    Business_Review_Word.append(WordList)    
    

For three number following a noun, the first number is positive review number, the second number is neutral review number and third is negative review number. 

In [74]:
Word

['pizza 20 6 2;food 7 2 1;sauce 11 0 1;service 7 0 3;crust 4 2 0;garlic 4 2 1',
 'pizza 7 2 0',
 'pizza 15 5 1;sauce 5 2 0;crust 2 4 0;price 5 2 0;dough 1 5 0',
 'pizza 6 2 1',
 'pizza 43 10 2;food 12 2 1;cheese 8 4 0;sauce 13 4 0;service 10 3 3;salad 6 2 0;crust 13 5 1;delivery 2 4 0;staff 7 0 1;bread 8 0 1;pie 6 3 0;pepperoni 2 3 1;sausage 6 2 0;garlic 6 0 1;spaghetti 6 4 0',
 '',
 'pizza 9 4 0;food 13 4 3;cheese 3 2 2;sauce 10 4 0;service 14 2 2;salad 5 5 2;restaurant 8 4 0;menu 2 4 1;staff 4 3 0;price 5 0 2;bread 6 5 1;dinner 3 3 1;meal 5 4 1;pasta 12 2 0;meatball 2 5 0;sausage 4 2 1;kitchen 3 5 1',
 'pizza 32 10 5;food 13 7 2;cheese 8 9 1;service 9 2 5;crust 3 5 1;chicken 3 4 0;delivery 8 11 3;price 3 3 0;flavor 5 1 1;pepperoni 2 4 0;fry 5 3 1;steak 3 2 3;hoagie 1 4 3;wing 4 2 0',
 'pizza 2 5 2',
 '',
 'pizza 8 2 0',
 '',
 '',
 'pizza 9 3 0;food 11 4 0;service 7 1 0',
 '',
 'pizza 15 5 1',
 'pizza 6 3 1',
 '',
 'pizza 180 42 5;food 55 16 3;cheese 27 13 0;sauce 18 17 3;service 58 8

In [75]:
Review_Suggestion=pd.DataFrame([ID,Word]).transpose()

In [76]:
Review_Suggestion=Review_Suggestion.rename(columns={0: "business_id", 1: "Positive_Negative_Percentage"})

In [77]:
Review_Suggestion

,business_id,Positive_Negative_Percentage
0,ZkzutF0P_u0C0yTulwaHkA,pizza 20 6 2;food 7 2 1;sauce 11 0 1;service 7...
1,39lLJK_rrYY2NYomSsQdUA,pizza 7 2 0
2,0y6alZmSLnPzmG5_kP5Quw,pizza 15 5 1;sauce 5 2 0;crust 2 4 0;price 5 2...
3,_Kp1IPTi17wBywYOd30raA,pizza 6 2 1
4,crBw_Hq3gCkppirgjnIl4A,pizza 43 10 2;food 12 2 1;cheese 8 4 0;sauce 1...
...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,pizza 39 9 4;food 7 1 3;cheese 3 3 1;sauce 3 3...
1556,ZZGs9idRiKRHue2RyXw9uA,pizza 6 1 2
1557,v8GgrqLTs_NZZl2tkyvOgA,food 5 4 1
1558,qWcaGio9JsqrvgmQiIRtIQ,pizza 32 5 1;sauce 4 2 1;crust 13 2 1;delivery...


Check, for business_id ZkzutF0P_u0C0yTulwaHkA, count review from pizza:

In [78]:
Target_Review=Word_Review_Matrix_New[Word_Review_Matrix_New["business_id"]=="ZkzutF0P_u0C0yTulwaHkA"]
Target_Review

,review_id,business_id,stars,pizza,food,cheese,sauce,service,salad,restaurant,...,doughy,cooking,diner,fee,wing,topping,dressing,olive,serving,seasoning
484,u6g7Pt7bBeL4-lQF0c0Usg,ZkzutF0P_u0C0yTulwaHkA,5,1.0,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185,Z-GyaOWtaABjF7uFlX4Dkg,ZkzutF0P_u0C0yTulwaHkA,5,1.0,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2825,FJ0_NqoOSVQdqAs1KGY0uw,ZkzutF0P_u0C0yTulwaHkA,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2951,-QxcD7S70uEo90kJfoiwSw,ZkzutF0P_u0C0yTulwaHkA,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3399,tuMfNsmEHoSZWAqB9RPvPQ,ZkzutF0P_u0C0yTulwaHkA,4,1.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
3921,o1_OXrwOKj5fZ7bPTUmD3A,ZkzutF0P_u0C0yTulwaHkA,5,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4272,IdhVlHpx0m5oy9f0o8Tm2w,ZkzutF0P_u0C0yTulwaHkA,5,1.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4779,FJUQiFu4E5k6ZdCsE4QCog,ZkzutF0P_u0C0yTulwaHkA,4,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4934,QQ4OuEuG0VoU2VIN_FyJbw,ZkzutF0P_u0C0yTulwaHkA,5,1.0,1.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5029,7M9xuZh2-5UrAGlO8sLcgA,ZkzutF0P_u0C0yTulwaHkA,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
Target_Review["pizza"].value_counts()

 1.0    20
 0.0     6
-1.0     2
Name: pizza, dtype: int64

No problem

The table above show every business the positive, negative, neutral reviews number towards a noun such as pizza.

Find the top 3 Negative or Positive Words by business:

In [80]:
pizza_business_review=pd.read_csv("data/Split_Review.csv")
pizza_business_review.iloc[0:10]

,Unnamed: 0,Unnamed: 0.1,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Clean_Review
0,0,0,hfXVxrlD6fr1kXfz6sR13Q,7nIzgNF7YzI-UAyYr3y9hw,J9f-9Prw2YVM-fiZqv2fmQ,2,0,1,1,Parking is tough to get here so I don't get to...,2013-09-04 04:35:27,"['parking tough get not get go much', ' also g..."
1,1,1,EDQpLLFiAxeSxWI5ipwzBQ,SkueY7hVsyXmYItELoodGw,lpel3pRFAzlow7u2TJMz4Q,4,3,0,0,The server had an extensive knowledge of wines...,2015-04-28 19:28:12,['server extensive knowledge wine various menu...
2,2,2,RWmT90rP7wYo57dG1cyvww,n3ahpmnEXsZYj3nbVMoUmg,Zja7ykq3icDzkN20XX6j1w,5,1,0,1,Fast Forward\n\nEvery trip from NYC to Pitt in...,2013-07-12 23:08:35,"['fast forward', 'every trip nyc pitt includes..."
3,3,3,0jduztItUhOh3FzKGjMxZQ,lVbPEaNUxFjwUydjentrYw,hr0vxeTzlHx6fTpSSmu2AA,5,1,0,0,By far my favorite Cleveland/Akron restaurant!...,2013-12-07 01:09:23,"['far favorite clevelandakron restaurant', ' g..."
4,4,4,sxfmzhMvZ8LlI1x4fhNjKg,SQKRGzDa7qd8qHb13Z2-Hg,F4kjKxXXyn3e-BYVb1mWcg,4,0,0,0,All food is good here I usually get the Italia...,2014-10-27 01:42:40,"['food good usually get italian sub pizza', ' ..."
5,5,5,Mp7qXWexqoSqKbNh7dYhJg,jAYIkCIqhDgQcp7PG8z_vA,J9f-9Prw2YVM-fiZqv2fmQ,5,1,0,0,Fries are cooked to perfection....very crispy....,2015-05-26 07:45:02,"['fry cooked perfection', 'very crispy', ' por..."
6,6,6,5e_JUyKVFpYxl9qHp1MG1Q,w1LWJ4l7jDeMNmmgIUKZyg,7xHbnGo4MXFkBm6gYSu_xg,2,1,0,0,"The pizza was great, but the guys that make it...",2018-05-02 18:55:11,['pizza great guy make scratching heads taking...
7,7,7,D2oYk9ipsvsYP_ls0aL9kg,9IA3P3DxrZcoAl29IYOloA,eU_713ec6fTGNO4BegRaww,4,0,0,1,"It's on Mt. Washington, but no view. The insi...",2009-08-03 20:21:18,"['mt', ' washington no view', ' inside look li..."
8,8,8,CKHk7l_VriSEqPbG1PSLTw,gP0DqgbBsPTHmu_RXW8TxA,epWC7CGRbNHkXM7G2ulYzQ,3,0,0,0,There is a large dine-in area that is child fr...,2017-09-08 23:39:53,['large dinein area child friendly arcade game...
9,9,9,iFKNo8gEgKwuWTmRPmJ-MQ,SbsUVsP2gkQhJf4L2q4kjg,ZWq8Fq13JPDAqt7rPjSwcQ,5,0,0,0,"Great breakfast! Sunrise special: two eggs, a ...",2016-05-19 14:03:48,"['great breakfast', ' sunrise special two eggs..."


In [81]:
Positive_Word=pd.read_csv("data/Positive_Word.csv")
Negative_Word=pd.read_csv("data/Negative_Word.csv")
Important_Positive_Word=pd.read_csv("data/Important_Positive_Word.csv")
Important_Negative_Word=pd.read_csv("data/Important_Negative_Word.csv")

In [82]:
Positive_Word_List=Positive_Word["Word"].tolist()
Negative_Word_List=Negative_Word["Word"].tolist()
Important_Positive_Word_List=Important_Positive_Word["Word"].tolist()
Important_Negative_Word_List=Important_Negative_Word["Word"].tolist()

In [83]:
replace_words={
 "n't":"not",
 "don't":"do not",
 "aren't":"are not",
 "can't":"can not",
 "couldn't":"could not",
 "didn't":"did not",
 "doesn't":"does not",
 "hadn't":"had not",
 "hasn't":"has not",
 "haven't":"have not",
 "isn't":"is not",
 "mightn't":"might not",
 "mustn't":"must not",
 "needn't":"need not",
 "shan't":"shall not",
 "shouldn't":"should not",
 "wasn't":"was not",
 "weren't":"were not",
 "won't":"will not",
 "wouldn't":"would not"}

In [84]:
stop_words = stopwords.words('english')
stop_words = [i for i in stop_words if i not in ["not",'no','never',"n't","don't","aren't","can't","couldn't", "didn't","doesn't","hadn't","hasn't","haven't","isn't","mightn't","mustn't","needn't","shan't","shouldn't","wasn't","weren't","won't","wouldn't"]]
stop_words



['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

# Attention: Code running for long time below:

In [85]:
Clean_Review=[]
Review=pizza_business_review["text"]
for i in range(len(Review)):
    review=Review[i]
    review=review.lower()
    Split_Review=review.replace("!",".")
    Split_Review=Split_Review.replace("?",".")
    Split_Review=Split_Review.replace(";",".")
    Split_Review=Split_Review.replace("\n",".")
    Split_Review=Split_Review.split(" ")
    clean_review=[]
    for i in Split_Review:
        if i in set(replace_words.keys()):
            clean_review.append(replace_words.get(i))
        else:
            clean_review.append(i)
    clean_review=(" ".join(clean_review)).split(" ")
    review=[i for i in clean_review if i not in stop_words]     
    review=[lmtzr.lemmatize(i) for i in review]
    review=" ".join(review)
    review="".join([i for i in review if i not in '"#$%&\'()*+,-/:<=>@[\\]^_`{|}~\'\'``-'])
    review = re.sub(' +',' ',review)
    Split_Review=review.split(".")
    Split_Review=[i for i in Split_Review if i not in [" ",""]]
    Split_Review
    Clean_Review.append(Split_Review)

In [86]:
r,c=pizza_business_review.shape
c=len(Review_Word)
Word_Count_Positive=[[0 for _ in range(c)] for _ in range(r)]
Word_Count_Negative=[[0 for _ in range(c)] for _ in range(r)]
Word_Count_Important_Positive=[[0 for _ in range(c)] for _ in range(r)]
Word_Count_Important_Negative=[[0 for _ in range(c)] for _ in range(r)]
for i in range(r):
    for j in range(c):
        Target=Review_Word[j]
        Review=Clean_Review[i]
        Target_Review=[]
        for Small_Review in Review:
            if Target in Small_Review.split(" "):
                Target_Review.append(Small_Review)
        Target_Review=" ".join(Target_Review)
        
        if any(c.isalpha() for c in Target_Review)==True:
            Target_Review=Target_Review.split(" ")
            Word_Count_Positive[i][j]=[word for word in Positive_Word_List if word in Target_Review]
            Word_Count_Negative[i][j]=[word for word in Negative_Word_List if word in Target_Review]
            Word_Count_Important_Positive[i][j]=[word for word in Important_Positive_Word_List if word in Target_Review]
            Word_Count_Important_Negative[i][j]=[word for word in Important_Negative_Word_List if word in Target_Review]
        

These four matrix show the positive, negative, important positive and important negative word of i th review to j th noun words. For example:

In [87]:
Word_Count_Positive[0][25]

['great', 'huge', 'recommend', 'super']

In [88]:
Clean_Review[0]

['parking tough get not get go much',
 ' also get little cautious sometimes frequently multiple questionable looking characters',
 'pros',
 ' legendary fry used super huge gave great value buck',
 ' good burgers',
 ' drink look like lot different kind alcohol',
 'cons',
 ' customer service',
 ' hahaha',
 ' not here',
 ' get fart one twins working there ahead pack',
 ' dirty messy tables crap everywhere',
 ' many questionable character here',
 ' someone asked wanted buy stolen jewelry eating',
 ' im totally honest too',
 ' french fry portion gotten smaller',
 ' used need two basket large fry need one',
 ' really expensive hot dogs',
 ' strict limit ketchup',
 ' no im not joking',
 ' 2 large fries 1 anything less',
 'i recommend getting legendary fry least recommend egg burger get go go light out']

In [89]:
Review_Word[25]

'fry'

The word is "fry", the small review contains "service" is:



    

 ' legendary fry used super huge gave great value buck',

 ' used need two basket large fry need one',
 
 ' 2 large fries 1 anything less',
 
 'i recommend getting legendary fry least recommend egg burger get go go light out'
 
 
 
 
    
and the positive words is 


'great', 'huge', 'recommend', 'super'


No problem.

In [90]:
Word_Count_Negative[1][28]

['complaint', 'uncomfortable']

In [91]:
Review_Word[28]

'wine'

In [92]:
Clean_Review[1]

['server extensive knowledge wine various menu selection clear top priority',
 ' pizza dessert amazing',
 ' complaint chair uncomfortable think could add comfortable seating give wine bar feel',
 ' otherwise everything amazing']

In [93]:
Word_Count_Important_Positive[1][28]

['comfortable']

In [94]:
Word_Count_Important_Negative[1][28]

['uncomfortable']

No problem again

Both comfortable and uncomfortable occur in Word_Count_Important_Positive and Word_Count_Important_Negative word list. 

Therefore, the four matrix above find the positive, negative, important positive, important negative words in review i towards important noun j.

First combine the four matrix by words.

In [95]:
Review_Word

['pizza',
 'food',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'menu',
 'delivery',
 'staff',
 'price',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'server',
 'flavor',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'location',
 'fry',
 'meatball',
 'waitress',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'manager',
 'style',
 'mushroom',
 'pepper',
 'dough',
 'dessert',
 'soup',
 'steak',
 'onion',
 'bartender',
 'bacon',
 'kitchen',
 'hoagie',
 'fan',
 'parking',
 'mozzarella',
 'employee',
 'water',
 'potato',
 'waiter',
 'entree',
 'oil',
 'cream',
 'serve',
 'shrimp',
 'fish',
 'parmesan',
 'spinach',
 'beef',
 'egg',
 'brunch',
 'ice',
 'chef',
 'breadstick',
 'breakfast',
 'cake',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'bbq',
 'music',
 'lasagna',
 'cocktail',
 'eggplant',
 'gnocchi',
 'butter',
 'ravioli',
 'taco',
 'margherita',
 'chocolate',
 'coffee',
 'takeout',
 'management',
 'building',


In [96]:
Word1=[]
Word2=[]
Word3=[]
Count1=[]
Count2=[]
Count3=[]
Sum=[]
for i in range(len(Review_Word)):
    A=[word for word in [row[i] for row in Word_Count_Positive] if word!=0 and word!=[]]
    B=[item for sublist in A for item in sublist]
    Word=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False).index[0:3].tolist()
    Count=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False)[0][0:3].tolist()
    Sum.append(Word_Review_Matrix_New[Review_Word[i]].tolist().count(1))
    Word1.append(Word[0])
    Word2.append(Word[1])
    Word3.append(Word[2])
    Count1.append(Count[0])
    Count2.append(Count[1])
    Count3.append(Count[2])
Positive_by_word=pd.DataFrame([Review_Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum]).transpose()

Positive_by_word=Positive_by_word.rename(columns={0: "Word", 1: "Word1", 2: "Count1",3: "Word2",4: "Count2",5: "Word3",6: "Count3",7: "Sum"})

Positive_by_word

,Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum
0,pizza,good,8277,best,5561,great,5491,27807
1,food,good,5315,great,4473,delicious,1398,15948
2,cheese,good,1244,like,985,fresh,613,6056
3,sauce,good,1597,like,1132,sweet,1099,7104
4,service,great,3674,good,3036,friendly,1912,10950
...,...,...,...,...,...,...,...,...
105,topping,good,441,fresh,438,like,356,2257
106,dressing,good,247,fresh,157,like,128,1121
107,olive,like,116,fresh,111,good,107,694
108,serving,good,63,like,57,great,43,443


This table show you the top 3 positive words towards a target word and their count, and the sum of all positive reviews mention them.

Repeat with other three matrix

In [97]:
Word1=[]
Word2=[]
Word3=[]
Count1=[]
Count2=[]
Count3=[]
Sum=[]
for i in range(len(Review_Word)):
    A=[word for word in [row[i] for row in Word_Count_Negative] if word!=0 and word!=[]]
    B=[item for sublist in A for item in sublist]
    Word=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False).index[0:3].tolist()
    Count=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False)[0][0:3].tolist()
    Sum.append(Word_Review_Matrix_New[Review_Word[i]].tolist().count(-1))
    Word1.append(Word[0])
    Word2.append(Word[1])
    Word3.append(Word[2])
    Count1.append(Count[0])
    Count2.append(Count[1])
    Count3.append(Count[2])
    
Negative_by_word=pd.DataFrame([Review_Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum]).transpose()

Negative_by_word=Negative_by_word.rename(columns={0: "Word", 1: "Word1", 2: "Count1",3: "Word2",4: "Count2",5: "Word3",6: "Count3",7: "Sum"})

Negative_by_word

,Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum
0,pizza,bad,824,ill,513,wrong,489,2814
1,food,bad,550,terrible,243,wrong,225,2139
2,cheese,bad,91,wrong,56,disappointed,44,688
3,sauce,bad,107,wrong,67,die,58,777
4,service,terrible,604,bad,581,poor,485,2584
...,...,...,...,...,...,...,...,...
105,topping,wrong,42,missing,31,charged,24,304
106,dressing,wrong,20,missing,14,die,13,143
107,olive,crushed,11,bad,10,hate,7,69
108,serving,refused,4,bad,4,complaint,4,75


As you can see, if we count from positive and negative word list. The most frequent are words like "good", "bad", which are too broad. This is why we design important positive and negative word list for raising suggestion.

In [98]:
Word1=[]
Word2=[]
Word3=[]
Count1=[]
Count2=[]
Count3=[]
Sum=[]
for i in range(len(Review_Word)):
    A=[word for word in [row[i] for row in Word_Count_Important_Positive] if word!=0 and word!=[]]
    B=[item for sublist in A for item in sublist]
    Word=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False).index[0:3].tolist()
    Count=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False)[0][0:3].tolist()
    Sum.append(Word_Review_Matrix_New[Review_Word[i]].tolist().count(1))
    Word1.append(Word[0])
    Word2.append(Word[1])
    Word3.append(Word[2])
    Count1.append(Count[0])
    Count2.append(Count[1])
    Count3.append(Count[2])
    
Important_Positive_by_word=pd.DataFrame([Review_Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum]).transpose()

Important_Positive_by_word=Important_Positive_by_word.rename(columns={0: "Word", 1: "Word1", 2: "Count1",3: "Word2",4: "Count2",5: "Word3",6: "Count3",7: "Sum"})

Important_Positive_by_word

,Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum
0,pizza,delicious,2808,fresh,1644,free,692,27807
1,food,delicious,1398,fresh,621,cheap,211,15948
2,cheese,fresh,613,delicious,530,cheap,86,6056
3,sauce,fresh,691,delicious,661,free,55,7104
4,service,delicious,367,clean,183,fresh,145,10950
...,...,...,...,...,...,...,...,...
105,topping,fresh,438,delicious,158,free,41,2257
106,dressing,fresh,157,delicious,106,healthy,12,1121
107,olive,fresh,111,delicious,55,free,7,694
108,serving,delicious,24,fresh,22,healthy,5,443


In [99]:
Word1=[]
Word2=[]
Word3=[]
Count1=[]
Count2=[]
Count3=[]
Sum=[]
for i in range(len(Review_Word)):
    A=[word for word in [row[i] for row in Word_Count_Important_Negative] if word!=0 and word!=[]]
    B=[item for sublist in A for item in sublist]
    if B!=[]:
        Word=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False).index[0:3].tolist()
        Count=pd.DataFrame.from_dict(Counter(B), orient='index').sort_values(0,ascending=False)[0][0:3].tolist()
        Sum.append(Word_Review_Matrix_New[Review_Word[i]].tolist().count(-1))
        Word1.append(Word[0])
        Count1.append(Count[0])
        if len(Word)>1:
            Word2.append(Word[1])
            Count2.append(Count[1])
        else:
            Word2.append(" ")
            Count2.append(0)
        if len(Word)>2:
            Word3.append(Word[2])
            Count3.append(Count[2])
        else:
            Word3.append(" ")
            Count3.append(0)
    else:
            Word1.append(" ")
            Count1.append(0)
            Word2.append(" ")
            Count2.append(0)
            Word3.append(" ")
            Count3.append(0)
            Sum.append(Word_Review_Matrix_New[Review_Word[i]].tolist().count(-1))
Important_Negative_by_word=pd.DataFrame([Review_Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum]).transpose()

Important_Negative_by_word=Important_Negative_by_word.rename(columns={0: "Word", 1: "Word1", 2: "Count1",3: "Word2",4: "Count2",5: "Word3",6: "Count3",7: "Sum"})

Important_Negative_by_word

,Word,Word1,Count1,Word2,Count2,Word3,Count3,Sum
0,pizza,rude,138,snob,111,sick,56,2814
1,food,rude,127,sick,46,starving,25,2139
2,cheese,stingy,20,boring,15,rude,7,688
3,sauce,boring,20,stingy,18,sick,6,777
4,service,rude,192,shitty,19,unprofessional,18,2584
...,...,...,...,...,...,...,...,...
105,topping,boring,8,rude,7,stingy,5,304
106,dressing,boring,2,insulting,1,rude,1,143
107,olive,insult,1,distracted,1,snob,1,69
108,serving,rude,4,disorganized,1,outrageous,1,75


It is easier to raise suggestion for them.

In [100]:
Important_Negative_by_word.to_csv("data/Suggestion_Important_Negative_by_word.csv")
Important_Positive_by_word.to_csv("data/Suggestion_Important_Positive_by_word.csv")
Negative_by_word.to_csv("data/Suggestion_Negative_by_word.csv")
Positive_by_word.to_csv("data/Suggestion_Positive_by_word.csv")

No problem

Find other information we want in suggestion in Shiny app. 

In [101]:
Food_P_N_Ratio=pd.merge(Positive_by_word, Negative_by_word, how='left', on='Word')

In [102]:
Food_P_N_Ratio

,Word,Word1_x,Count1_x,Word2_x,Count2_x,Word3_x,Count3_x,Sum_x,Word1_y,Count1_y,Word2_y,Count2_y,Word3_y,Count3_y,Sum_y
0,pizza,good,8277,best,5561,great,5491,27807,bad,824,ill,513,wrong,489,2814
1,food,good,5315,great,4473,delicious,1398,15948,bad,550,terrible,243,wrong,225,2139
2,cheese,good,1244,like,985,fresh,613,6056,bad,91,wrong,56,disappointed,44,688
3,sauce,good,1597,like,1132,sweet,1099,7104,bad,107,wrong,67,die,58,777
4,service,great,3674,good,3036,friendly,1912,10950,terrible,604,bad,581,poor,485,2584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,topping,good,441,fresh,438,like,356,2257,wrong,42,missing,31,charged,24,304
106,dressing,good,247,fresh,157,like,128,1121,wrong,20,missing,14,die,13,143
107,olive,like,116,fresh,111,good,107,694,crushed,11,bad,10,hate,7,69
108,serving,good,63,like,57,great,43,443,refused,4,bad,4,complaint,4,75


In [103]:
Food_P_N_Ratio["PositiveNegativeRatio"]=Food_P_N_Ratio["Sum_x"]/Food_P_N_Ratio["Sum_y"]
Food_P_N_Ratio["All_Sum"]=(len(Word_Review_Matrix_New["stars"])-Word_Review_Matrix_New.isna().sum()[3:]).tolist()

In [104]:
Food_P_N_Ratio.sort_values("PositiveNegativeRatio")

,Word,Word1_x,Count1_x,Word2_x,Count2_x,Word3_x,Count3_x,Sum_x,Word1_y,Count1_y,Word2_y,Count2_y,Word3_y,Count3_y,Sum_y,PositiveNegativeRatio,All_Sum
92,attitude,like,38,great,34,good,32,174,bad,52,rude,29,poor,16,198,0.878788,616
35,manager,like,112,care,74,great,69,611,rude,149,wrong,61,bad,35,576,1.06076,2276
88,management,good,42,care,39,like,35,241,poor,49,rude,29,bad,21,200,1.205,716
49,parking,easy,98,good,79,free,73,533,difficult,54,limited,43,pain,35,272,1.95956,1594
51,employee,friendly,151,nice,98,like,95,589,rude,79,bad,26,dirty,19,274,2.14964,1361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,eggplant,good,73,delicious,55,like,42,423,die,7,wrong,5,bad,4,33,12.8182,692
7,crust,good,1350,like,1036,great,795,6406,fired,89,bad,61,wrong,38,495,12.9414,9920
78,cocktail,great,88,good,74,delicious,40,426,disappointed,6,ill,6,bloody,5,32,13.3125,710
62,spinach,good,86,fresh,76,like,51,555,wrong,8,disappointed,7,bad,6,38,14.6053,1079


In [105]:
n=len(Food_P_N_Ratio["Word"].tolist())
Drop=[]
for i in range(n):
    if Food_P_N_Ratio["Word"].tolist()[i] not in New_Noun_results["Food"].tolist():
        Drop.append(i)
Food_P_N_Ratio=Food_P_N_Ratio.drop(Drop)
Food_P_N_Ratio

,Word,Word1_x,Count1_x,Word2_x,Count2_x,Word3_x,Count3_x,Sum_x,Word1_y,Count1_y,Word2_y,Count2_y,Word3_y,Count3_y,Sum_y,PositiveNegativeRatio,All_Sum
9,beer,good,1249,great,1210,nice,379,4014,bad,66,limited,58,wrong,33,270,14.8667,6133
28,wine,good,382,great,371,nice,197,1669,limited,27,ill,19,disappointed,18,150,11.1267,2826
30,sausage,good,263,like,179,great,145,1379,die,11,disappointed,11,wrong,11,120,11.4917,2863
45,bacon,good,141,like,88,delicious,81,803,die,14,wrong,11,crazy,8,93,8.63441,1644
50,mozzarella,fresh,442,good,129,like,91,948,crushed,16,die,11,fired,10,82,11.561,1585
59,shrimp,good,115,like,50,great,42,467,bad,8,disappointing,4,horrible,4,49,9.53061,933
60,fish,good,115,like,60,great,59,450,bad,8,disappointing,6,wrong,6,52,8.65385,809
71,marinara,good,107,like,73,delicious,53,487,die,8,wrong,7,bad,5,58,8.39655,947
73,spaghetti,good,105,like,85,delicious,40,440,bad,8,disappointed,7,die,5,49,8.97959,876
74,lettuce,fresh,96,good,80,like,61,348,bad,13,rotten,8,unfortunately,6,90,3.86667,964


In [106]:
Food_P_N_Ratio=Food_P_N_Ratio[["Word","Sum_x","Sum_y","PositiveNegativeRatio","All_Sum"]].sort_values("PositiveNegativeRatio")

In [107]:
Food_P_N_Ratio

,Word,Sum_x,Sum_y,PositiveNegativeRatio,All_Sum
74,lettuce,348,90,3.86667,964
95,caesar,210,43,4.88372,454
104,wing,1862,272,6.84559,3205
99,tiramisu,256,32,8,462
71,marinara,487,58,8.39655,947
45,bacon,803,93,8.63441,1644
60,fish,450,52,8.65385,809
73,spaghetti,440,49,8.97959,876
59,shrimp,467,49,9.53061,933
84,margherita,379,39,9.71795,788


The table show the positive and negative review num ratio of the food in Noun suggestion. Not that the noun in Noun suggestion is not included into noun in Review suggestion. Because some are not significant in t test in review analysis.

In [108]:
Food_P_N_Ratio.to_csv("data/Suggestion_Positive_Negative_Food_Ratio.csv")

In [109]:
ID=[]
Review_Num=[]
Average_Review_Star_Rating=[]
for i in business_review.tolist():
    ID.append(i)
    Review_Num.append(len(pizza_review[pizza_review["business_id"]==i]["stars"]))
    Average_Review_Star_Rating.append(round(pizza_review[pizza_review["business_id"]==i]["stars"].mean(),3))

In [110]:
Review_NUM=pd.DataFrame([ID,Review_Num,Average_Review_Star_Rating]).transpose()
Review_NUM=Review_NUM.rename(columns={0: "business_id", 1: "Review_Num",2: "Average_Review_Star_Rating"})
Review_NUM

,business_id,Review_Num,Average_Review_Star_Rating
0,ZkzutF0P_u0C0yTulwaHkA,32,3.875
1,39lLJK_rrYY2NYomSsQdUA,11,2.455
2,0y6alZmSLnPzmG5_kP5Quw,21,4.524
3,_Kp1IPTi17wBywYOd30raA,15,2.267
4,crBw_Hq3gCkppirgjnIl4A,63,4.429
...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,60,3.7
1556,ZZGs9idRiKRHue2RyXw9uA,10,3
1557,v8GgrqLTs_NZZl2tkyvOgA,12,2.75
1558,qWcaGio9JsqrvgmQiIRtIQ,39,4.231


In [111]:
Other_Information=pizza_business[["business_id","name",'address',"city",'latitude','longitude','stars']]
Other_Information

,business_id,name,address,city,latitude,longitude,stars
0,ZkzutF0P_u0C0yTulwaHkA,Lelulos Pizzeria,311 Unity Center Rd,Pittsburgh,40.489996,-79.779288,4.0
1,39lLJK_rrYY2NYomSsQdUA,Marco's Pizza,24335 Chagrin Blvd,Cleveland,41.465789,-81.506349,2.5
2,0y6alZmSLnPzmG5_kP5Quw,J J's Pizza,20542 Lorain Rd,Cleveland,41.448341,-81.847644,4.5
3,_Kp1IPTi17wBywYOd30raA,Pizza Hut,8609 University Blvd,Pittsburgh,40.507268,-80.222632,2.0
4,crBw_Hq3gCkppirgjnIl4A,Famous Yeti's Pizza,971 N Page St,Madison,42.926526,-89.222592,4.5
...,...,...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,Blaze Fast-Fire'd Pizza,2610 N Prospect Ave,Urbana-Champaign,40.147209,-88.254940,3.5
1556,ZZGs9idRiKRHue2RyXw9uA,Marco's Pizza,5877 Broadview Rd,Cleveland,41.403398,-81.689779,3.0
1557,v8GgrqLTs_NZZl2tkyvOgA,Cleveland Clinic Cafeteria,"9500 Euclid Ave, Bldg C, Fl 1",Cleveland,41.501753,-81.621448,3.0
1558,qWcaGio9JsqrvgmQiIRtIQ,Country Style Pizza,536 Dewey Ave,Pittsburgh,40.356300,-80.110600,4.0


In [112]:
Suggestion=pd.merge(Business_Suggestion,Noun_Suggestion,how='left', on='business_id')
Suggestion=pd.merge(Suggestion,Review_Suggestion,how='left', on='business_id')
Suggestion=pd.merge(Suggestion,Other_Information,how='left', on='business_id')
Suggestion=pd.merge(Suggestion,Review_NUM,how='left', on='business_id')
Suggestion

,business_id,Alcohol,BikeParking,GoodForKids,NoiseLevel,RestaurantsGoodForGroups,RestaurantsReservations,WiFi,RestaurantsDelivery,BusinessParking,...,Avoid,Positive_Negative_Percentage,name,address,city,latitude,longitude,stars,Review_Num,Average_Review_Star_Rating
0,ZkzutF0P_u0C0yTulwaHkA,0,1,1,1,1,1,0,0,0,...,shrimp fish marinara spaghetti caesar noodle t...,pizza 20 6 2;food 7 2 1;sauce 11 0 1;service 7...,Lelulos Pizzeria,311 Unity Center Rd,Pittsburgh,40.489996,-79.779288,4.0,32,3.875
1,39lLJK_rrYY2NYomSsQdUA,0,0,1,1,0,0,0,0,0,...,shrimp fish marinara spaghetti lettuce caesar ...,pizza 7 2 0,Marco's Pizza,24335 Chagrin Blvd,Cleveland,41.465789,-81.506349,2.5,11,2.455
2,0y6alZmSLnPzmG5_kP5Quw,0,1,1,1,1,0,0,0,0,...,spaghetti lettuce caesar noodle tea broccoli c...,pizza 15 5 1;sauce 5 2 0;crust 2 4 0;price 5 2...,J J's Pizza,20542 Lorain Rd,Cleveland,41.448341,-81.847644,4.5,21,4.524
3,_Kp1IPTi17wBywYOd30raA,1,0,1,1,1,0,0,0,1,...,shrimp fish marinara spaghetti lettuce caesar ...,pizza 6 2 1,Pizza Hut,8609 University Blvd,Pittsburgh,40.507268,-80.222632,2.0,15,2.267
4,crBw_Hq3gCkppirgjnIl4A,1,1,1,1,1,0,1,0,1,...,shrimp fish noodle tea broccoli wing coke patt...,pizza 43 10 2;food 12 2 1;cheese 8 4 0;sauce 1...,Famous Yeti's Pizza,971 N Page St,Madison,42.926526,-89.222592,4.5,63,4.429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,Qtv6iGDAHBmAKBgl40JgLw,0,0,1,0,1,0,0,0,0,...,shrimp fish spaghetti lettuce caesar noodle te...,pizza 39 9 4;food 7 1 3;cheese 3 3 1;sauce 3 3...,Blaze Fast-Fire'd Pizza,2610 N Prospect Ave,Urbana-Champaign,40.147209,-88.254940,3.5,60,3.7
1556,ZZGs9idRiKRHue2RyXw9uA,0,0,1,1,1,0,0,0,0,...,shrimp fish marinara spaghetti lettuce caesar ...,pizza 6 1 2,Marco's Pizza,5877 Broadview Rd,Cleveland,41.403398,-81.689779,3.0,10,3
1557,v8GgrqLTs_NZZl2tkyvOgA,0,0,1,0,1,0,1,1,0,...,shrimp fish marinara spaghetti caesar noodle t...,food 5 4 1,Cleveland Clinic Cafeteria,"9500 Euclid Ave, Bldg C, Fl 1",Cleveland,41.501753,-81.621448,3.0,12,2.75
1558,qWcaGio9JsqrvgmQiIRtIQ,0,1,1,1,1,0,0,0,1,...,shrimp fish marinara spaghetti lettuce noodle ...,pizza 32 5 1;sauce 4 2 1;crust 13 2 1;delivery...,Country Style Pizza,536 Dewey Ave,Pittsburgh,40.356300,-80.110600,4.0,39,4.231


In [113]:
Suggestion.to_csv("data/Suggestion.csv")